In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau

from skimage.feature import graycomatrix, graycoprops
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, random
import cv2
import glob as gb
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score, classification_report
from pathlib import Path
from PIL import Image
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, Conv1D, MaxPool1D
from tensorflow.keras.layers import Dense, Input, Dropout, concatenate


In [4]:
trainData=[]
trainLabels=[]

for root, _, files in os.walk('/content/drive/MyDrive/DatasetMain-20240306T050200Z-001/Data4/Training/notumor'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'notumor' in image_path:
        label = 0
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        trainData.append(image_array)
        trainLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/content/drive/MyDrive/DatasetMain-20240306T050200Z-001/Data4/Training/glioma'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'glioma' in image_path:
        label = 1
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        trainData.append(image_array)
        trainLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/content/drive/MyDrive/DatasetMain-20240306T050200Z-001/Data4/Training/pituitary'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'pituitary' in image_path:
        label = 2
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        trainData.append(image_array)
        trainLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/content/drive/MyDrive/DatasetMain-20240306T050200Z-001/Data4/Training/meningioma'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'meningioma' in image_path:
        label = 3
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        trainData.append(image_array)
        trainLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")

In [5]:
testData=[]
testLabels=[]

for root, _, files in os.walk('/content/drive/MyDrive/DatasetMain-20240306T050200Z-001/Data4/Testing/notumor'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'notumor' in image_path:
        label = 0
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        testData.append(image_array)
        testLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/content/drive/MyDrive/DatasetMain-20240306T050200Z-001/Data4/Testing/glioma'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'glioma' in image_path:
        label = 1
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        testData.append(image_array)
        testLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/content/drive/MyDrive/DatasetMain-20240306T050200Z-001/Data4/Testing/pituitary'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'pituitary' in image_path:
        label = 2
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        testData.append(image_array)
        testLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/content/drive/MyDrive/DatasetMain-20240306T050200Z-001/Data4/Testing/meningioma'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'meningioma' in image_path:
        label = 3
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        testData.append(image_array)
        testLabels.append(label)

In [6]:
from keras.utils import to_categorical
X_train = trainData
y_train = trainLabels
X_test = testData
y_test = testLabels

y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

X_val, X_test = train_test_split(X_test, train_size=.3, random_state=20)
y_val, y_test = train_test_split(y_test, train_size=.3, random_state=20)

y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)


print(f"Shape of images in X_train: {X_train.shape}")
print(f"Shape of images in X_test: {X_test.shape}")
print(f"Shape of images in X_val: {X_val.shape}")
print(f"Shape of images in y_train: {y_train.shape}")
print(f"Shape of images in y_test: {y_test.shape}")
print(f"Shape of images in y_val: {y_val.shape}")

Shape of images in X_train: (5736, 224, 224, 3)
Shape of images in X_test: (918, 224, 224, 3)
Shape of images in X_val: (393, 224, 224, 3)
Shape of images in y_train: (5736, 4)
Shape of images in y_test: (918, 4)
Shape of images in y_val: (393, 4)


In [7]:
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)

In [8]:
print("Training Set Shape after preprocesssing = ",X_train.shape)
print("Validation Set Shape after preprocesssing= ",X_val.shape)
print("Test Set Shape after preprocesssing= ",X_test.shape)

Training Set Shape after preprocesssing =  (5736, 224, 224, 3)
Validation Set Shape after preprocesssing=  (393, 224, 224, 3)
Test Set Shape after preprocesssing=  (918, 224, 224, 3)


In [9]:
model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4,activation='softmax')


])

In [10]:
from tensorflow.keras.layers import GRU, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout


In [11]:
model = keras.models.Sequential([
    Conv2D(filters=64, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Conv2D(filters=192, kernel_size=(5, 5), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Conv2D(filters=384, kernel_size=(3, 3), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding="same"),
    BatchNormalization(),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding="same"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    # Flatten the output of CNN layers before passing it to GRU
   # Flatten the output of CNN layers before passing it to GRU
# Flatten the output of CNN layers before passing it to GRU
keras.layers.Reshape((-1, 128)),
    GRU(128),
    Dense(4, activation='softmax')
])


In [12]:
model.compile(optimizer ='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop= EarlyStopping(monitor='val_loss',patience=2)

In [14]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.001)

In [21]:
import time
start = time.time()
epochs =40
ThisModel = model.fit(X_train,y_train,epochs=epochs,validation_data = (X_val,y_val),batch_size=64,verbose=2,callbacks=[learning_rate_reduction])
print("Total time: ", time.time() - start, "seconds")

Epoch 1/40
90/90 - 4s - loss: 0.0933 - accuracy: 0.9716 - val_loss: 0.1938 - val_accuracy: 0.9542 - lr: 0.0010 - 4s/epoch - 45ms/step
Epoch 2/40
90/90 - 4s - loss: 0.0924 - accuracy: 0.9709 - val_loss: 0.1270 - val_accuracy: 0.9746 - lr: 0.0010 - 4s/epoch - 41ms/step
Epoch 3/40
90/90 - 4s - loss: 0.0699 - accuracy: 0.9800 - val_loss: 0.1846 - val_accuracy: 0.9542 - lr: 0.0010 - 4s/epoch - 40ms/step
Epoch 4/40
90/90 - 4s - loss: 0.0786 - accuracy: 0.9766 - val_loss: 0.1433 - val_accuracy: 0.9746 - lr: 0.0010 - 4s/epoch - 41ms/step
Epoch 5/40
90/90 - 4s - loss: 0.0821 - accuracy: 0.9745 - val_loss: 0.6307 - val_accuracy: 0.8601 - lr: 0.0010 - 4s/epoch - 44ms/step
Epoch 6/40
90/90 - 4s - loss: 0.0971 - accuracy: 0.9697 - val_loss: 0.1248 - val_accuracy: 0.9695 - lr: 0.0010 - 4s/epoch - 41ms/step
Epoch 7/40
90/90 - 4s - loss: 0.0955 - accuracy: 0.9707 - val_loss: 0.2152 - val_accuracy: 0.9262 - lr: 0.0010 - 4s/epoch - 41ms/step
Epoch 8/40
90/90 - 4s - loss: 0.1632 - accuracy: 0.9517 - val_

In [22]:
model.evaluate(X_test, y_test)

29/29 [==============================] - 0s 14ms/step - loss: 0.2398 - accuracy: 0.9390


[0.23984551429748535, 0.9389978051185608]